In [1]:
import os
import pandas as pd

In [2]:
results_path = r'C:\Users\huda\Downloads\including_milp\including_milp'

In [3]:
categories = os.listdir(results_path)
results = dict()
for category in categories:
    files = os.listdir(results_path + '/' + category)
    results[category] = dict()
    for file in files:
        file_content = pd.read_csv(results_path + '/' + category + '/' + file, names=['name','optimization_status','fitness_value','lower_bound', 'runtime', 'result_vector1', 'result_vector2', 'result_vector3', 'peak_cpu', 'peak_ram', 'resource_history', 'best_result_history'], sep=';')
        df = pd.DataFrame(file_content)
        results[category][file[8:-4]] = dict()
        for index, row in df.iterrows():
            if row['name'].startswith('Error'):
                print(f'Error in {row["name"]}')
                continue
            results[category][file[8:-4]][row['name']] = {
                'name': row['name'],
                'fitness': row['fitness_value'],
                'lower_bound': row['lower_bound'],
                'runtime': min(300, row['runtime']),
                'peak_cpu': row['peak_cpu'],
                'peak_ram': row['peak_ram']
            }


Error in Error on instance 0_BehnkeGeiger_60_workers.fjs using solver ortools: local variable 'lower_bound' referenced before assignment
Error in Error on instance 0_BehnkeGeiger_60_workers.fjs using solver ortools: local variable 'lower_bound' referenced before assignment


In [4]:
results['no_callback']

{'cplex_cp': {'6_Fattahi_20_workers.fjs': {'name': '6_Fattahi_20_workers.fjs',
   'fitness': 1154,
   'lower_bound': 884,
   'runtime': 300,
   'peak_cpu': 0.0,
   'peak_ram': 0.0},
  '6_Fattahi_14_workers.fjs': {'name': '6_Fattahi_14_workers.fjs',
   'fitness': 538,
   'lower_bound': 538,
   'runtime': 28.336,
   'peak_cpu': 0.0,
   'peak_ram': 0.0},
  '5_Kacem_4_workers.fjs': {'name': '5_Kacem_4_workers.fjs',
   'fitness': 11,
   'lower_bound': 10,
   'runtime': 300,
   'peak_cpu': 0.0,
   'peak_ram': 0.0},
  '5_Kacem_3_workers.fjs': {'name': '5_Kacem_3_workers.fjs',
   'fitness': 7,
   'lower_bound': 7,
   'runtime': 0.863,
   'peak_cpu': 0.0,
   'peak_ram': 0.0},
  '4_ChambersBarnes_10_workers.fjs': {'name': '4_ChambersBarnes_10_workers.fjs',
   'fitness': 897,
   'lower_bound': 651,
   'runtime': 300,
   'peak_cpu': 0.0,
   'peak_ram': 0.0},
  '3_DPpaulli_9_workers.fjs': {'name': '3_DPpaulli_9_workers.fjs',
   'fitness': 2012,
   'lower_bound': 1274,
   'runtime': 300,
   'peak_cp

In [5]:
comparison = dict()
for category in results:
    for solver in results[category]:
        for instance in results[category][solver]:
            if results[category][solver][instance]['name'] not in comparison:
                comparison[results[category][solver][instance]['name']] = dict()
            if category not in comparison[results[category][solver][instance]['name']]:
                comparison[results[category][solver][instance]['name']][category] = []
            comparison[results[category][solver][instance]['name']][category].append((solver, results[category][solver][instance]['fitness'], results[category][solver][instance]['runtime'], results[category][solver][instance]['lower_bound']))

In [6]:
differences_fitness = dict()
differences_runtime = dict()
differences_lower_bound = dict()
for instance in comparison:
    no_callback = comparison[instance]['no_callback']
    with_callback = comparison[instance]['with_callback']
    for i in range(len(no_callback)):
        if no_callback[i][0] != with_callback[i][0]:
            print('mismatch')
        else:
            if no_callback[i][0] not in differences_fitness:
                differences_fitness[no_callback[i][0]] = []
                differences_runtime[no_callback[i][0]] = []
                differences_lower_bound[no_callback[i][0]] = []
            #print(f'{instance}:\nSolver: {no_callback[i][0]}\nFitness\nNo Callback: {no_callback[i][1]} - With Callback: {with_callback[i][1]} = {no_callback[i][1]-with_callback[i][1]}')
            differences_fitness[no_callback[i][0]].append(no_callback[i][1]-with_callback[i][1])
            differences_runtime[no_callback[i][0]].append(no_callback[i][2]-with_callback[i][2])
            differences_lower_bound[no_callback[i][0]].append(no_callback[i][3]-with_callback[i][3])



In [7]:
print('Fitness')
for solver in differences_fitness:
    tie = 0
    no_callback_win = 0
    with_callback_win = 0
    for difference in differences_fitness[solver]:
        if difference == 0:
            tie += 1
        elif difference < 0:
            no_callback_win += 1
        else:
            with_callback_win += 1
    print(f'{solver}: Ties: {tie}, No Callback: {no_callback_win}, With Callback: {with_callback_win}')

Fitness
cplex_cp: Ties: 29, No Callback: 0, With Callback: 0
hexaly: Ties: 18, No Callback: 7, With Callback: 4
ortools: Ties: 7, No Callback: 12, With Callback: 9
cplex_lp: Ties: 11, No Callback: 1, With Callback: 3
gurobi: Ties: 14, No Callback: 1, With Callback: 0


In [8]:
print('Runtime')
for solver in differences_runtime:
    tie = 0
    no_callback_win = 0
    with_callback_win = 0
    for difference in differences_runtime[solver]:
        if difference == 0:
            tie += 1
        elif difference < 0:
            no_callback_win += 1
        else:
            with_callback_win += 1
    print(f'{solver}: Ties: {tie}, No Callback: {no_callback_win}, With Callback: {with_callback_win}')

Runtime
cplex_cp: Ties: 25, No Callback: 1, With Callback: 3
hexaly: Ties: 29, No Callback: 0, With Callback: 0
ortools: Ties: 24, No Callback: 4, With Callback: 0
cplex_lp: Ties: 5, No Callback: 4, With Callback: 6
gurobi: Ties: 5, No Callback: 10, With Callback: 0


In [9]:
print('Lower Bound')
for solver in differences_lower_bound:
    tie = 0
    no_callback_win = 0
    with_callback_win = 0
    for difference in differences_lower_bound[solver]:
        if difference == 0:
            tie += 1
        elif difference < 0:
            no_callback_win += 1
        else:
            with_callback_win += 1
    print(f'{solver}: Ties: {tie}, No Callback: {with_callback_win}, With Callback: {no_callback_win}') # max is better, so switch

Lower Bound
cplex_cp: Ties: 29, No Callback: 0, With Callback: 0
hexaly: Ties: 29, No Callback: 0, With Callback: 0
ortools: Ties: 16, No Callback: 6, With Callback: 6
cplex_lp: Ties: 13, No Callback: 2, With Callback: 0
gurobi: Ties: 13, No Callback: 2, With Callback: 0
